In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import h5py
import json

def colorbar(mappable, ax=plt.gca()):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
hbar_amu_THz = 6.35 # Å^2
THz__eV = 4.136e-3

In [ ]:
basedir='../examples/Graphene'

# EPCs at k-point

In [ ]:
with h5py.File(basedir +'/el-ph/el-ph-Nq200-K-3.hdf5', 'r') as f:
    mesh_qpoints = f['ph']['qpoints'][()]
    qvecs = f['ph']['qpointsCart'][()]
    mesh_frequencies = f['ph']['omega'][()] / THz__eV
    
    mesh_epskq = f['el']['eps_q_0'][()]
    
    mesh_g2 = f['el-ph']['g2_0'][:,:,3:5,3:5] # only read CB and VB
    Kvec = f['el-ph']['g2_0'].attrs['kvec']

In [ ]:
mC = 12.010700
qx = qvecs[:,0]
qy = qvecs[:,1]
q_unit = 1.

fig, axs = plt.subplots(2, 2, figsize=(3*3, 3*3), sharex='col', sharey='row')

ax = axs[0,0]
lq = (hbar_amu_THz/(2*mC*mesh_frequencies[:,0]))
im = ax.tricontourf(qx/q_unit, qy/q_unit, np.sqrt(mesh_g2[:, 0, 1, 1]/lq), levels=20)
ax.scatter(0.0, 0.0, color='white', s=3)

ax.set_aspect('equal')
colorbar(im, ax=ax)
ax.text(-0.063, 0.047, 'TA', fontsize=14)
ax.set_xlim((-0.05,0.05))
ax.set_ylim((-0.05,0.05))

ax = axs[0,1]
lq = (hbar_amu_THz/(2*mC*mesh_frequencies[:,1]))
im = ax.tricontourf(qx/q_unit, qy/q_unit, np.sqrt(mesh_g2[:, 1, 1, 1]/lq), levels=20)
ax.scatter(0.0, 0.0, color='white', s=3)

ax.set_aspect('equal')
cbar = colorbar(im, ax=ax)
cbar.set_label(r'$|g^\lambda_{CB}(\vec{q})| / \ell^\lambda_{\vec{q}}$ [ eV/Å]', fontsize=12)
ax.text(-0.063, 0.047, 'LA', fontsize=14)
ax.set_xlim((-0.05,0.05))
ax.set_ylim((-0.05,0.05))

ax = axs[1,0]
lq = (hbar_amu_THz/(2*mC*mesh_frequencies[:,2]))
im = ax.tricontourf(qx/q_unit, qy/q_unit, np.sqrt(mesh_g2[:, 2, 1, 1]/lq), levels=20)
ax.scatter(0.0, 0.0, color='white', s=3)

ax.set_aspect('equal')
colorbar(im, ax=ax)
ax.text(-0.063, 0.047, 'LO', fontsize=14)
ax.set_xlim((-0.05,0.05))
ax.set_ylim((-0.05,0.05))

ax = axs[1,1]
lq = (hbar_amu_THz/(2*mC*mesh_frequencies[:,3]))
im = ax.tricontourf(qx/q_unit, qy/q_unit, np.sqrt(mesh_g2[:, 3, 1, 1]/lq), levels=20)
ax.scatter(0.0, 0.0, color='white', s=3)

ax.set_aspect('equal')
cbar = colorbar(im, ax=ax)
cbar.set_label(r'$|g^\lambda_{CB}(\vec{q})| / \ell^\lambda_{\vec{q}}$ [ eV/Å]', fontsize=12)
ax.text(-0.063, 0.047, 'TO', fontsize=14)
ax.set_xlim((-0.05,0.05))
ax.set_ylim((-0.05,0.05))

plt.setp(axs[-1, :], xlabel=r'$q_x$ [$2\pi/a$]')
plt.setp(axs[:, 0],  ylabel=r'$q_y$ [$2\pi/a$]')

plt.tight_layout()
plt.show()
#plt.savefig(basedir +'/el-ph/graphene-dftb-elph-Mmatrix.pdf')

# EPCs along band-path

In [ ]:
with open(basedir +'/el-ph/ephline.json') as jfile:
    data = json.load(jfile)
data = json.loads(data)

indices = np.array(data['highSymIndices'])
indices = indices-1
indices[0] = 0
labels = data['highSymLabels']
qpoints = np.array(data['wavevectorCoordinatesCart'])
energies = np.array(data['energies'])
frequencies = np.array(data['frequencies'])
eigenvectors = np.array(data['eigenvectors_real']) + 1j*np.array(data['eigenvectors_imaginary'])
epcs = np.array(data['epcs'])
data['numBands']

q_dist_dftb = [0.,]
for i in range(1, qpoints.shape[0]):
    q_dist_dftb.append(np.linalg.norm(qpoints[i,:]-qpoints[i-1,:]))

In [ ]:
nb = 3 # start with VB

fig, axs = plt.subplots(3, 6, figsize=(3*6, 2*3), sharex='col')

for b in range(2):
    for m in range(6):
        ax1 = axs[b,m]
        ax1.scatter(np.cumsum(q_dist_dftb), epcs[:,m,nb+b,nb+b], 
                         marker='o', linewidth=0, alpha=0.75, zorder=2)
        
        ax1.set_xticks(np.cumsum(q_dist_dftb)[indices])
        ax1.set_xticklabels(labels)
        ax1.set_title(r'ibnd=%i, jbnd=%i, imode=%i' % (nb+b, nb+b, m))
        
axs[0,0].set_ylabel(r'$g$ [eV]', fontsize=16)
axs[1,0].set_ylabel(r'$g$ [eV]', fontsize=16)


for m in range(6):
    ax1 = axs[2,m]
    ax1.scatter(np.cumsum(q_dist_dftb), frequencies[:, m], 
                     marker='o', linewidth=0, alpha=0.75, zorder=2)    

    ax1.set_xticks(np.cumsum(q_dist_dftb)[indices])
    ax1.set_xticklabels(labels)
    ax1.set_title(r'imode=%i' % (m))

axs[2,0].set_ylabel(r'$\omega$ [eV]', fontsize=16)

plt.tight_layout()
plt.show()
#plt.savefig(basedir +'/el-ph/ephline.pdf')